In [2]:

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
import extract_correct_csv
scaler = StandardScaler()


## logistic regression with sklearn

In [47]:
valid_subject = extract_correct_csv.extract_only_valid_subject()
valid_subject.remove(49)
valid_subject

[2, 4, 10, 41, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55]

In [60]:
columns = ['subject', 'feature','fold', 'train accuracy', 'test accuracy']
results = pd.DataFrame(columns = columns)
index = 1
for x in valid_subject:



    types_ = ['hr', 'eda', 'pupil']
    for type_ in types_:
        X = pd.read_csv('data/features_4_2/'+type_+'/45.csv')
        X = X[48:]
        X = scaler.fit_transform(X)
        # creating train and test
        X = pd.DataFrame(X)
        X = X.reset_index().drop(columns=('index'))



        sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

        for i, (train_index, test_index) in enumerate(sss.split(X, y)):
            y = pd.DataFrame(y)
            y = y.reset_index().drop(columns=('index'))

            N_train = len(train_index)
            X_train = X.iloc[train_index, :]
            y_train =y.iloc[train_index, :]

            X_test = X.iloc[test_index, :]
            y_test =y.iloc[test_index, :]

            clf = LogisticRegression(random_state=123,max_iter=10000)
            clf.fit(X_train, np.squeeze(np.array(y_train)))

            train_ = clf.score(X_train, y_train)
            test_ = clf.score(X_test, y_test)
            dict_ = {'subject': x, 'feature': type_,'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
            results = pd.concat([results, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
            index = index + 1

## accuracy logistic classifier sklearn monophysio

In [64]:
reg_mono = results
reg_mono

,subject,feature,fold,train accuracy,test accuracy
0,2,hr,0,0.584270,0.521739
1,2,hr,1,0.662921,0.304348
2,2,hr,2,0.595506,0.608696
3,2,eda,0,0.662921,0.608696
4,2,eda,1,0.685393,0.652174
...,...,...,...,...,...
139,55,eda,1,0.640449,0.695652
140,55,eda,2,0.662921,0.521739
141,55,pupil,0,0.741573,0.695652
142,55,pupil,1,0.764045,0.652174


In [65]:
columns = ['subject', 'fold']

In [69]:
mean_regr=reg_mono.drop(columns=columns).groupby(by='feature', as_index=False).mean()
var_regr=reg_mono.drop(columns=columns).groupby(by='feature', as_index=False).var()

## mean accuracy over all subjs logistic classifier sklearn monophysio

In [70]:
mean_regr

,feature,train accuracy,test accuracy
0,eda,0.685627,0.626812
1,hr,0.627575,0.563406
2,pupil,0.713717,0.573370


## var over all subjs logistic classifier sklearn monophysio


In [67]:
var_regr

,feature,train accuracy,test accuracy
0,eda,0.003865,0.009639
1,hr,0.001665,0.006995
2,pupil,0.002192,0.008419


# Multi physio classifier

In [77]:
subjects = extract_correct_csv.extract_only_valid_subject()
subjects

[2, 4, 10, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55]

In [79]:

columns_multi = ['subject','fold', 'train accuracy', 'test accuracy']
reg_multi = pd.DataFrame(columns = columns_multi)
index = 1

for sub in subjects:

    string_sub = extract_correct_csv.read_correct_subject_csv(sub)

    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]


    X1 = pd.read_csv('data/features_4_2/hr/'+str(sub)+'.csv')
    X1 = pd.DataFrame(scaler.fit_transform(X1))
    X2 = pd.read_csv('data/features_4_2/eda/'+str(sub)+'.csv')
    X2 = pd.DataFrame(scaler.fit_transform(X2))
    X3 = pd.read_csv('data/features_4_2/pupil/'+str(sub)+'.csv')
    X3 = pd.DataFrame(scaler.fit_transform(X3))
    X = pd.concat([X1, X2, X3], axis=1)
    X = X[48:]

    X = pd.DataFrame(X)
    X = X.reset_index().drop(columns=('index'))

    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        classifier = LogisticRegression(max_iter=10000,random_state=123)
        clf.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': sub, 'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        reg_multi = pd.concat([reg_multi, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
        index = index + 1

## accuracy logistic classifier sklearn

In [80]:
reg_multi

,subject,fold,train accuracy,test accuracy
0,2,0,0.730337,0.608696
1,2,1,0.853933,0.521739
2,2,2,0.786517,0.521739
3,4,0,0.719101,0.521739
4,4,1,0.730337,0.434783
5,4,2,0.685393,0.608696
6,10,0,0.707865,0.565217
7,10,1,0.741573,0.478261
8,10,2,0.685393,0.652174
9,41,0,0.853933,0.478261


In [ ]:
columns = ['subject', 'fold']

## accuracy logistic classifier sklearn mean by subj

In [83]:
grouped_df = reg_multi.groupby(by=['subject'], as_index=False).mean()
grouped_df

/tmp/ipykernel_52844/1239913191.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_df = reg_multi.groupby(by=['subject'], as_index=False).mean()


,subject,train accuracy,test accuracy
0,2,0.790262,0.550725
1,4,0.711610,0.521739
2,10,0.711610,0.565217
3,41,0.823970,0.550725
4,43,0.831461,0.579710
5,44,0.805243,0.594203
6,45,0.853933,0.652174
7,46,0.749064,0.565217
8,47,0.805243,0.521739
9,48,0.838951,0.594203


## mean accuracy over all subjs logistic classifier sklearn

In [85]:
grouped_df.drop(columns=['subject']).mean()

train accuracy    0.783873
test accuracy     0.581415
dtype: float64

## var over all subjs logistic classifier sklearn

In [87]:
grouped_df.drop(columns=['subject']).var()

train accuracy    0.002141
test accuracy     0.001520
dtype: float64

In [32]:
print("test accuracy all physio\nmean: ",np.mean(np.array(mean_test)))
print('var: ',np.var(np.array(mean_test)))

test accuracy all physio
mean:  0.5959079283887468
var:  0.0072867131952302775


# MLP

In [20]:
from sklearn.neural_network import MLPClassifier
valid_subject = extract_correct_csv.extract_only_valid_subject()
valid_subject.remove(50)
valid_subject.remove(51)

mean_MLP_train = []
mean_MLP_test = []
for sub in valid_subject:
    string_sub = extract_correct_csv.read_correct_subject_csv(sub)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]


    X1 = pd.read_csv('data/features_4_2/hr/'+str(sub)+'.csv')
    X1 = pd.DataFrame(scaler.fit_transform(X1))
    X2 = pd.read_csv('data/features_4_2/eda/'+str(sub)+'.csv')
    X2 = pd.DataFrame(scaler.fit_transform(X2))
    X3 = pd.read_csv('data/features_4_2/pupil/'+str(sub)+'.csv')
    X3 = pd.DataFrame(scaler.fit_transform(X3))

    X = pd.concat([X1, X2, X3], axis=1)
    X = X[48:]
    X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=123, stratify=y)
    clf = MLPClassifier(hidden_layer_sizes=100, max_iter=700, learning_rate='adaptive', random_state=123).fit(X_train, y_train)
    mean_MLP_train.append(clf.score(X_train, y_train))
    mean_MLP_test.append(clf.score(X_test, y_test))

In [21]:
np.mean(np.array(mean_MLP_train))

1.0

In [22]:
np.mean(np.array(mean_MLP_test))

0.6627450980392156

In [23]:
np.std(np.array(mean_MLP_test))

0.07583952786593612

## Logistic regression with morph level

In [1]:
# extract morph level functions
def extract_cs(df):
    cs1, cs2 = int(df[df.shock == True].picName.unique()[0][5]), int(df[df.shock == True].picName.unique()[1][5])
    return cs1, cs2


morph_pos = 11
morph_value = 15


def extract_threat_level(df):
    MORPH_POS = 11
    MORPH_VALUE = 15
    threat_person = []
    cs1_pic, cs2_pic = extract_cs(df)

    for i in df.iterrows():
        nome = i[1].picName
        try:
            if nome[5] == str(cs1_pic) or nome[5] == str(cs2_pic):
                threat_person.append(6)
                continue
            elif int(nome[5]):
                threat_person.append(1)
                continue
        except:
            if (int(nome[MORPH_POS]) == cs1_pic) or (int(nome[MORPH_POS]) == cs2_pic):
                if nome[MORPH_VALUE] == '2':
                    threat_person.append(5)

                elif nome[MORPH_VALUE] == '4':
                    threat_person.append(4)

                elif nome[MORPH_VALUE] == '6':
                    threat_person.append(3)

                elif nome[MORPH_VALUE] == '8':
                    threat_person.append(2)
                    continue
            else:
                if nome[MORPH_VALUE] == '2':
                    threat_person.append(2)

                elif nome[MORPH_VALUE] == '4':
                    threat_person.append(3)

                elif nome[MORPH_VALUE] == '6':
                    threat_person.append(4)

                elif nome[MORPH_VALUE] == '8':
                    threat_person.append(5)
    return threat_person

In [ ]:
subjects = extract_correct_csv.extract_only_valid_subject()


In [8]:
columns = ['subject', 'feature', 'fold', 'train accuracy', 'test accuracy']
results = pd.DataFrame(columns = columns)

for x in subjects:
    string_sub = extract_correct_csv.read_correct_subject_csv(x)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    df_ = df_[48:]
    y = np.array(list([int (d > 2) for d in df_['rating']]))


    img = np.array(extract_threat_level(df_))
    morph_level = np.zeros((img.size, img.max()))
    morph_level[np.arange(img.size), img-1] = 1
    X = morph_level.astype(int)
    X = pd.DataFrame(X)
    X = X.reset_index().drop(columns='index')

    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        clf = LogisticRegression(random_state=123,max_iter=10000)
        clf.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': x, 'feature': 'morph level' ,'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        results = pd.concat([results, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)

In [11]:
results.groupby(by='subject').mean(numeric_only=True)

,train accuracy,test accuracy
subject,,
2,0.794007,0.811594
4,0.670412,0.666667
10,0.696629,0.782609
41,0.771536,0.623188
43,0.868914,0.855072
44,0.707865,0.579710
45,0.921348,0.913043
46,0.655431,0.637681
47,0.910112,0.840580


## Mean over all subjects

In [14]:
results.drop(columns=['subject', 'fold']).mean(numeric_only=True)

train accuracy    0.797312
test accuracy     0.774936
dtype: float64